In [11]:
class GeneratorConfig:
    def __init__(self, entity_singular, entity_plural, prompt):
        self.entity_singular = entity_singular
        self.entity_plural = entity_plural
        self.prompt = prompt

In [5]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

import openai
openai.organization = organization
openai.api_key = api_key

gpt_model_name = "gpt-3.5-turbo"

def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message['content']

In [10]:
world_ped_prompt = """
Improvise a fictional fantasy world geography for a D&D 5e campaign setting.

The response should be formatted as markdown, in the following format.

# <World Name>

## <First Continent Name>

<First paragraph - Description of the continent's geography. Should desribe the major topographical features, regions, climates, and biomes. MUST NOT describe the inhabitants>

<Second paragraph - Description of the economic activity, degree of settlement, and races of the inhabitants for the regions in the continent.>

## <Second Continent Name>

<similar>

## <Third Continent Name>

<similar>
"""

world = basic_completion(world_ped_prompt, max_tokens=1024)
print(world)

# Fynthia

## Aeloria

Aeloria is a continent that is largely covered in dense forests and mountain ranges, with few open plains or deserts. The northwestern region of the continent is dominated by the towering Snowpeak Mountains, which are covered in snow and ice year-round. To the south of the mountains, there are vast coniferous forests known as the Wyvernwood, which is home to a variety of exotic and dangerous creatures. The central region of Aeloria is dominated by the sprawling Glimmerwood, a magical forest said to be inhabited by fairies and other mysterious beings. At the eastern coast of the continent, there is a network of rivers and swamps known as the Marshlands, which are teeming with all sorts of amphibian and reptilian creatures.

The people of Aeloria are a diverse group, with a variety of races coexisting in relative peace. Elves and humans are the most common races here, and they are concentrated in the more settled regions along the coasts and rivers. Dwarves, gnomes

In [37]:
from mistletoe import Document

world_doc = Document(world)

raw_texts = [node.children[0].content for node in world_doc.children]

world_name = raw_texts[0]
world_name

i = 0
continent_raws = raw_texts[1:]
continents = []
while 3*i+2 < len(continent_raws):
    offset = 3*i
    name = continent_raws[offset]
    geography = continent_raws[offset + 1]
    economy = continent_raws[offset + 2]
    continents.append((name, geography, economy))
    i += 1

(world_name, continents)

('Fynthia',
 [('Aeloria',
   'Aeloria is a continent that is largely covered in dense forests and mountain ranges, with few open plains or deserts. The northwestern region of the continent is dominated by the towering Snowpeak Mountains, which are covered in snow and ice year-round. To the south of the mountains, there are vast coniferous forests known as the Wyvernwood, which is home to a variety of exotic and dangerous creatures. The central region of Aeloria is dominated by the sprawling Glimmerwood, a magical forest said to be inhabited by fairies and other mysterious beings. At the eastern coast of the continent, there is a network of rivers and swamps known as the Marshlands, which are teeming with all sorts of amphibian and reptilian creatures.',
   'The people of Aeloria are a diverse group, with a variety of races coexisting in relative peace. Elves and humans are the most common races here, and they are concentrated in the more settled regions along the coasts and rivers. Dwa

In [40]:
def continent_ped_prompt(continent): return f"""
Improvise the details of a continent's regions in a fantasy world for a D&D 5e campaign setting.

The continent's name is {(continent[0])}
{continent[1]}
{continent[2]}

The response should be formatted as markdown, in the following format.

# <Continent Name>

## <First Region Name>

<First paragraph - Description of the region's geography. Should describe geographical points of interest, climate, biome, and natural resources. MUST NOT describe the inhabitants.>

<Second paragraph - Description of the region's inhabitants. Should describe the races, major economic activities, and the major power factions.>

## <Second Region Name>

<similar>

...etc

"""

continent_response = basic_completion(continent_ped_prompt(continents[0]), max_tokens=1024)
print(continent_response)

# Aeloria

## Snowpeak Mountains

The Snowpeak Mountains are a massive range of towering peaks, stretching across the entire northwestern region of Aeloria. These mountains are covered in snow and ice year-round, and are home to a variety of creatures adapted to the harsh environment. The sparse vegetation found here is primarily used by the hardy hunting tribes who call the mountains home. 

The dwarves have found ample opportunities for mining here, digging into the mountain faces to extract metal and gemstones. They've also developed an extremely tough breed of goats, which they use to transport ores and metals from their mines.

## Wyvernwood

The sprawling Wyvernwood is a dense forest located to the south of the Snowpeak Mountains. The forest floor is dominated by massive conifers and rich undergrowth, making it difficult to traverse. In addition to the dense fauna, the Wyvernwood is home to a variety of magical creatures. 

A loose band of humans and elves have made their homes i

In [41]:
from mistletoe import Document

continent_doc = Document(continent_response)

raw_texts = [node.children[0].content for node in continent_doc.children]

continent_name = raw_texts[0]
continent_name

i = 0
region_raws = raw_texts[1:]
regions = []
while 3*i+2 < len(region_raws):
    offset = 3*i
    name = region_raws[offset]
    geography = region_raws[offset + 1]
    economy = region_raws[offset + 2]
    regions.append((name, geography, economy))
    i += 1

(continent_name, regions)

('Aeloria',
 [('Snowpeak Mountains',
   'The Snowpeak Mountains are a massive range of towering peaks, stretching across the entire northwestern region of Aeloria. These mountains are covered in snow and ice year-round, and are home to a variety of creatures adapted to the harsh environment. The sparse vegetation found here is primarily used by the hardy hunting tribes who call the mountains home.',
   "The dwarves have found ample opportunities for mining here, digging into the mountain faces to extract metal and gemstones. They've also developed an extremely tough breed of goats, which they use to transport ores and metals from their mines."),
  ('Wyvernwood',
   'The sprawling Wyvernwood is a dense forest located to the south of the Snowpeak Mountains. The forest floor is dominated by massive conifers and rich undergrowth, making it difficult to traverse. In addition to the dense fauna, the Wyvernwood is home to a variety of magical creatures.',
   'A loose band of humans and elves 

In [47]:
def region_ped_prompt(region): return f"""
Improvise the details of a region's settlements in a fantasy world for a D&D 5e campaign setting.

The region's name is {region[0]}
{region[1]}
{region[2]}

The response should be formatted as markdown, in the following format.

# Region Name

## <First Settlement Name>

<First paragraph - Description of the settlement's location, economy, and size>

<Second paragraph - Description of the settlement's politics>

## <Second Settlement Name>

<similar>

...etc

"""

region_response = basic_completion(region_ped_prompt(regions[0]), max_tokens=1024)
print(region_response)

# Snowpeak Mountains

## Frosthold

Frosthold is a small hunting village located in the heart of the Snowpeak Mountains. The village is home to a handful of hardy tribes who have learned to survive in the harsh environment. These tribes rely on hunting and gathering for their survival, using the sparse vegetation found in the area. Occasionally, they trade with passing caravan merchants who are willing to brave the treacherous terrain.

The tribes are fiercely independent and often engage in skirmishes with each other, but they all share a common enemy in the creatures that lurk in the mountains. 

## Ironhold

Ironhold is a dwarven mining town located deep within the Snowpeak Mountains. The dwarves have carved out an entire city within the mountain, complete with homes, shops, and even a temple to Moradin. The town's primary industry is mining, with the dwarves extracting metal and gemstones from the mountain faces. They've also developed their own breed of extremely tough goats, whic

In [43]:
region_response = basic_completion(region_ped_prompt(regions[1]), max_tokens=1024)
print(region_response)

# Wyvernwood

## Deepwood Village

Deepwood Village is the largest settlement in Wyvernwood, located in a clearing deep in the forest. The village is primarily made up of simple wooden structures, surrounded by tall wooden walls that protect against the dangerous creatures that roam the forest. The economy of Deepwood Village is centered around exotic creature hunting, with most of the village's inhabitants earning a living from selling the materials they gather from their hunts. Despite the dangers of their livelihood, the people of Deepwood Village are a close-knit community, relying on each other to survive in the forest.

Politically, Deepwood Village is led by a council of elders, made up of the most respected members of the community. The council makes decisions for the village as a whole, working to protect the people from the dangers of the forest while also ensuring that everyone has enough to eat. Conflict within the village is rare, as the people of Deepwood are bonded toget

In [44]:
region_response = basic_completion(region_ped_prompt(regions[2]), max_tokens=1024)
print(region_response)

# Glimmerwood

## Faerieland

Faerieland is a small collection of homes and shops built into the trees of the Glimmerwood. The people of Faerieland are known for their handcrafted goods that sell for high prices in bigger cities. The economy is largely run on bartering with other settlements, but some traders will occasionally travel to Faerieland to purchase goods to bring elsewhere. The settlement is home to primarily elves, who revere the fairies and often work with them on various enchantments to enhance their wares. 

Politically, Faerieland operates in a more communal fashion, with a council of village elders who make decisions for the group. Disagreements within the council are handled through discussion and debate, with a focus on maintaining harmony within the village. Punishments for crimes are usually handled through community service or a payment to the victim, and more serious crimes are rare due to the small size of the settlement and the tight-knit community. 

## Belleh

In [45]:
region_response = basic_completion(region_ped_prompt(regions[3]), max_tokens=1024)
print(region_response)

# Marshlands

## Riverhaven

Nestled on the banks of a wide river, Riverhaven is a bustling Halfling community consisting of a network of boats and floating platforms. The Halflings here make their living from fishing, hunting the local swamp creatures, and collecting plants and herbs for medicinal and magical use. The settlement is of moderate size, with around 200 permanent residents and another 100 or so visitors passing through at any given time.

Politically, Riverhaven is largely independent, but the Halflings here maintain trade relationships with seafaring merchants, allowing them to access a wider range of goods. The community is ruled by an elected council of elders, who moderate disputes and oversee the settlement's defense.

## Mudtown

Located on the edge of a large swamp, Mudtown is a scrappy Halfling settlement resembling a shanty town of ramshackle huts and tents. The Halflings here eke out a living through scavenging, stealing, and occasional trading with Riverhaven or

In [50]:
oneshot_ped_prompt = """
Located on the edge of a large swamp, Mudtown is a scrappy Halfling settlement resembling a shanty town of ramshackle huts and tents. The Halflings here eke out a living through scavenging, stealing, and occasional trading with Riverhaven or direct traders.

Politically, Mudtown is a place of anarchy, with no central authority or law enforcement. Instead, the strongest members of the community enforce their own version of justice, frequently resorting to violence to resolve disputes. Despite its precarious situation, Mudtown's inhabitants are fiercely independent and will not hesitate to defend their patch of swamp from incursion by outsiders.

Improvise the details of a one-shot adventure in Riverhaven for a D&D 5e setting.

The difficulty of the adventure should be tailored to a group of level 1 characters.

The complexity and linearity of the adventure should be tailored to new players, and should not take more than 3 hours.

The response should have the following format:

# <Campaign Name>

## <Antagonist's Name>

<Paragraph - A description of the antagonists acting in the adventure, including specific hostile NPCs>

## <Name of Exposition Act>

<Paragraph - A scene where the players' characters meet eachother and an NPC delivers the quest>

<Paragraph - Some other notable friendly NPCs in the setting which provide hooks to guide the players to the first encounter>

## <Name of Rising Action Act>

<Paragraph - A simple encounter, with terrain and enemy details>

<Paragraph - A scene after the first encounter with an obvious hook to guide the players to the final encounter>

## <Name of Climax Act>

<Paragraph - A more challenging final encounter, with terrain and enemy details>
"""

oneshot_response = basic_completion(oneshot_ped_prompt, max_tokens=2048)
print(oneshot_response)

# Swamp Skirmish

## Gatorgore Gang

The Gatorgore Gang, a group of unsavory sorts who make their living by poaching giant alligators and selling the hide, meat, and teeth for profit. They have recently begun encroaching on Mudtown and causing trouble for the Halflings. Led by a half-orc named Grimgor Gatorgore, the gang consists of a mixture of human, half-orc, and half-elf thugs.

## A Call for Help

As the players are walking through Riverhaven, they hear a commotion coming from the direction of the Halfling district. Running to see what's going on, they find a group of Halflings gathered around a man who appears to be their leader. He introduces himself as Kipper, the de facto leader of Mudtown, and asks the adventurers to help his people deal with the Gatorgore Gang's harassment.

## Friendly Faces

While in Mudtown, the players can interact with several friendly NPCs who could provide useful information and hooks for the upcoming encounters. These could include a friendly Halflin